In [1]:
import requests
from bs4 import BeautifulSoup
base_url = "https://www.monster.com/jobs/"

In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

С главной страницы раздела Browse Jobs собираем ссылки на штаты

In [3]:
states_links = []
r = requests.get(base_url)
soup = BeautifulSoup(r.text, 'html.parser')
box = soup.find("ul", class_="card-columns")
for a in box.find_all('a'):
    states_links.append(a['href'])
print(states_links[:3])

['https://www.monster.com/jobs/browse/l-alabama.aspx', 'https://www.monster.com/jobs/browse/l-alaska.aspx', 'https://www.monster.com/jobs/browse/l-arizona.aspx']


На странице штата требуется развернуть полный список городов

In [4]:
def selenium_click(link):
    # Используем селениум, чтобы раскрыть полный список городов
    driver = webdriver.Firefox(executable_path='/Users/froggityfrog/geckodriver')
    driver.get(link)
    elem = driver.find_element_by_link_text("View All")
    elem.click()
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    driver.quit()
    return soup

После разворачивания списка городов получим список ссылок на страницы вакансий городов

In [5]:
def get_cities(soup):
    #получаем список ссылок на города
    cities = []
    all_cities = soup.find('ul', class_="card-columns browse-all")
    all_h2 = all_cities.find_all('h2')
    for h2 in all_h2:
        cities.append(h2.a['href'])
    return cities

На странице вакансий города собираем ссылки на сами вакансии

In [6]:
def get_job_links(url):
    #получаем список ссылок на страницы вакансий
    r = requests.get(url)
    if r.ok:
        soup = BeautifulSoup(r.text, 'html.parser')
        links = []
        divs = soup.find_all("div", class_="jobTitle")
        for div in divs:
            links.append(div.a['href'])
        return links

На странице вакансии собираем описание

In [7]:
def get_text(link):
    r = requests.get(link)
    soup = BeautifulSoup(r.text, 'html.parser')
    div = soup.find("div", id="JobDescription")
    text = []
    for string in div.stripped_strings:
        text.append(string.strip().rstrip())
    return " ".join(text)

Собираем все вместе. Ссылки на штаты у нас уже имеются.

In [ ]:
with open('monster_job_descriptions.txt', 'w') as outfile:
    job_links = [] # сюда будем класть ссылки на вакансии
    for link in states_links[:1]:   # ссылки на штаты получены нами ранее
        soup = selenium_click(link) # кликаем по ссылке, получаем в ответ суп
        cities = get_cities(soup)   # вынимаем из супа ссылки на города
        
        #Переходим по каждой ссылке города и собираем ссылки на вакансии этого города
        for url in cities[:1]:      
            #Мы не знаем, сколько страниц вакансий в городе, так что повторяем, пока не закончатся страницы
            for i in range(1, 51):
                city_jobs = get_job_links(url+"?page="+str(i)) # проходимся по страницам
                if city_jobs != None:
                    job_links += city_jobs
        
    # На этом моменте у нас есть список ссылок на вакансии. 
    for link in job_links[:50]:
        text = get_text(link)      # собираем текст
        outfile.write(text+'\n')   # записываем текст в файл
        